<a href="https://colab.research.google.com/github/kazzastic/BreastCancer-Deep-Learning/blob/master/Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential

Using TensorFlow backend.


In [3]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape = (227,227,3))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
58892288/58889256 [==============================] - 2s 0us/step


In [37]:
'''
CLASSES = 2
x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)
x = Dropout(0.4)(x)
predictions = Dense(CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
'''


"\nCLASSES = 2\nx = base_model.output\nx = GlobalAveragePooling2D(name='avg_pool')(x)\nx = Dropout(0.4)(x)\npredictions = Dense(CLASSES, activation='softmax')(x)\nmodel = Model(inputs=base_model.input, outputs=predictions)\n"

In [0]:
base_model.trainable = False

In [0]:
add_model = Sequential()
add_model.add(base_model)
add_model.add(Conv2D(96, kernel_size=5, strides=1))
add_model.add(Activation('relu'))

add_model.add(Conv2D(256, kernel_size=3, strides=1))
add_model.add(Activation('relu'))

#add_model.add(Conv2D(96, kernel_size=3, strides=1))
#add_model.add(Activation('relu'))

#add_model.add(GlobalAveragePooling2D())
#add_model.add(Dropout(0.5))
add_model.add(Flatten())
add_model.add(Dense(2))
add_model.add(Activation('sigmoid'))


model = add_model
kazim = tf.keras.optimizers.Adam(learning_rate=0.001)
#tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

model.compile(loss='binary_crossentropy',
              optimizer=kazim,
              metrics=['accuracy'],
              )

In [0]:
for layer in base_model.layers:
    layer.trainable = False

In [19]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 3, 96)          1228896   
_________________________________________________________________
activation_6 (Activation)    (None, 3, 3, 96)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 1, 1, 256)         221440    
_________________________________________________________________
activation_7 (Activation)    (None, 1, 1, 256)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                

In [0]:
train_data_dir = '/content/drive/My Drive/Augmented '

In [21]:
# Pixel values rescaling from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1. / 255, validation_split=0.2)
batch_size = 32
img_width, img_height = 227, 227
# Retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        shuffle = True,
        batch_size=batch_size,
        #class_mode='binary',
        subset = 'training')

validation_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        shuffle = True,
        batch_size=batch_size,
        #class_mode='binary',
        subset = 'validation')

Found 7716 images belonging to 2 classes.
Found 1929 images belonging to 2 classes.


In [0]:
epochs = 100
train_samples = 7716
validation_samples = 1929

In [0]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
# Model saving callback

checkpointer = ModelCheckpoint(filepath='Breast.h5', monitor='val_acc', verbose=1, save_best_only=True)

# Early stopping
early_stopping = EarlyStopping(monitor='val_acc', verbose=1, patience=5)

history = model.fit_generator(
        train_generator,
        steps_per_epoch=32,
        epochs=epochs,
        validation_data = validation_generator,
        validation_steps = 32,
        callbacks=[checkpointer, early_stopping],
        verbose=1)

Epoch 1/100
32/32 [==============================] - 646s 20s/step - loss: 0.6185 - acc: 0.6704

Epoch 00001: val_acc improved from -inf to 0.67041, saving model to Breast.h5
32/32 [==============================] - 1029s 32s/step - loss: 0.7061 - acc: 0.6108 - val_loss: 0.6185 - val_acc: 0.6704
Epoch 2/100
32/32 [==============================] - 438s 14s/step - loss: 0.5966 - acc: 0.7012

Epoch 00002: val_acc improved from 0.67041 to 0.70117, saving model to Breast.h5
32/32 [==============================] - 685s 21s/step - loss: 0.6335 - acc: 0.6411 - val_loss: 0.5966 - val_acc: 0.7012
Epoch 3/100
32/32 [==============================] - 443s 14s/step - loss: 0.6042 - acc: 0.7061

Epoch 00003: val_acc improved from 0.70117 to 0.70605, saving model to Breast.h5
32/32 [==============================] - 688s 22s/step - loss: 0.6056 - acc: 0.6596 - val_loss: 0.6042 - val_acc: 0.7061
Epoch 4/100
32/32 [==============================] - 444s 14s/step - loss: 0.6720 - acc: 0.6230

Epoch 00